<a href="https://colab.research.google.com/github/Karthik0528/Data-Science---ML/blob/main/Fake%20news%20Detection/P4_FakeNewsDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IF 1 = its fake  &  0 = it is real



In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#printing the stopwords
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

# Load both CSV files
fake_df = pd.read_csv('/content/drive/MyDrive/fake.csv')
true_df = pd.read_csv('/content/drive/MyDrive/true.csv')

# Add label columns
fake_df['label'] = '1'
true_df['label'] = '0'

# Merge both DataFrames
merged_df = pd.concat([fake_df, true_df], ignore_index=True)

# Save merged file
merged_df.to_csv('/content/drive/MyDrive/news.csv', index=False)

print("✅ Merged file saved as news.csv")
print(merged_df.head())


✅ Merged file saved as news.csv
                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date label  
0  December 31, 2017     1  
1  December 31, 2017     1  
2  December 30, 2017     1  
3  December 29, 2017     1  
4  December 25, 2017     1  


In [ ]:
news_data = pd.read_csv('/content/drive/MyDrive/news.csv')

In [ ]:
news_data.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [ ]:
news_data.shape

(44898, 5)

In [ ]:
news_data.isnull().sum()

,0
title,0
text,0
subject,0
date,0
label,0


In [ ]:
x = news_data.drop(columns= 'label', axis=1)
y = news_data['label']

In [ ]:
print(x)
print(y)

                                                   title  \
0       Donald Trump Sends Out Embarrassing New Year’...   
1       Drunk Bragging Trump Staffer Started Russian ...   
2       Sheriff David Clarke Becomes An Internet Joke...   
3       Trump Is So Obsessed He Even Has Obama’s Name...   
4       Pope Francis Just Called Out Donald Trump Dur...   
...                                                  ...   
44893  'Fully committed' NATO backs new U.S. approach...   
44894  LexisNexis withdrew two products from Chinese ...   
44895  Minsk cultural hub becomes haven from authorities   
44896  Vatican upbeat on possibility of Pope Francis ...   
44897  Indonesia to buy $1.14 billion worth of Russia...   

                                                    text    subject  \
0      Donald Trump just couldn t wish all Americans ...       News   
1      House Intelligence Committee Chairman Devin Nu...       News   
2      On Friday, it was revealed that former Milwauk...       New

In [ ]:
#stemming
stem_port = PorterStemmer()

In [ ]:
def stemming(title):
  stemmed_title = re.sub('[^a-zA-Z]', ' ',title)
  stemmed_title = stemmed_title.lower()
  stemmed_title = stemmed_title.split()
  stemmed_title = [stem_port.stem(word) for word in stemmed_title if not word in stopwords.words('english')]
  stemmed_title = ' '.join(stemmed_title)
  return stemmed_title

In [ ]:
news_data['title'] = news_data['title'].apply(stemming)

In [ ]:
print(news_data['title'])

0        donald trump send embarrass new year eve messa...
1        drunk brag trump staffer start russian collus ...
2        sheriff david clark becom internet joke threat...
3            trump obsess even obama name code websit imag
4            pope franci call donald trump christma speech
                               ...                        
44893    fulli commit nato back new u approach afghanistan
44894         lexisnexi withdrew two product chines market
44895                        minsk cultur hub becom author
44896      vatican upbeat possibl pope franci visit russia
44897              indonesia buy billion worth russian jet
Name: title, Length: 44898, dtype: object


In [ ]:
X = news_data['title'].values
Y = news_data['label'].values

In [ ]:
print(X)

['donald trump send embarrass new year eve messag disturb'
 'drunk brag trump staffer start russian collus investig'
 'sheriff david clark becom internet joke threaten poke peopl eye' ...
 'minsk cultur hub becom author'
 'vatican upbeat possibl pope franci visit russia'
 'indonesia buy billion worth russian jet']


In [ ]:
print(Y)

[1 1 1 ... 0 0 0]


In [ ]:
print(Y.shape)

(44898,)


In [ ]:
#converting textual data into numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)


In [ ]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 411810 stored elements and shape (44898, 13206)>
  Coords	Values
  (0, 3339)	0.4152733860823366
  (0, 3404)	0.27228169452981993
  (0, 3731)	0.36585763997365384
  (0, 3949)	0.46737756630771704
  (0, 7335)	0.3506307808204068
  (0, 7866)	0.2355330109368446
  (0, 10341)	0.35617541225037314
  (0, 12010)	0.11874831789619875
  (0, 13104)	0.285897289994607
  (1, 1414)	0.39443884799795026
  (1, 2289)	0.41140099817201903
  (1, 3523)	0.45452116468198656
  (1, 5989)	0.3127247619981534
  (1, 9996)	0.28041269573559635
  (1, 11031)	0.4051439782731003
  (1, 11065)	0.3381297018822547
  (1, 12010)	0.11694584057077229
  (2, 999)	0.2899680869663155
  (2, 2144)	0.3679456237184158
  (2, 2881)	0.3373830961140794
  (2, 4067)	0.2910179557797341
  (2, 5960)	0.2946136857056743
  (2, 6194)	0.3180103625234714
  (2, 8601)	0.2337777620708223
  (2, 8836)	0.42473021917033454
  :	:
  (44893, 7866)	0.2316688299307463
  (44894, 2037)	0.3409101873703228
  (4489

In [ ]:
#splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state=2, stratify= Y)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(44898, 13206) (35918, 13206) (8980, 13206)


In [ ]:
#trainig the model
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

In [ ]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print('accuracy score of the training data:',training_data_accuracy)

accuracy score of the training data: 0.9599922044657275


In [ ]:
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('accuracy score of the testing data:',testing_data_accuracy)

accuracy score of the testing data: 0.9415367483296214


Making predictive system

In [ ]:
X_new = X_test[0]
prediction = model.predict(X_new)
print(prediction)

if prediction[0] == 0:
  print('the news is real')
else:
  print('the news is fake!')

[1]
the news is fake!


In [ ]:
print(Y_test[0])

1


Deploying the model

In [ ]:
import joblib
joblib.dump(model, "model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")

['vectorizer.pkl']

In [ ]:
import pickle

# Save model and vectorizer
pickle.dump(model, open('model.pkl', 'wb'))
pickle.dump(TfidfVectorizer, open('vectorizer.pkl', 'wb'))
